<h1><center>Contributions to presidential campaings in the state of Arizona in 2016</center></h1>

### Author: Julio Cárdenas-Rodríguez  
#### Contact: jdatascientist@gmail.com
### Source: [ftp://ftp.fec.gov/FEC/Presidential_Map/2016/P00000001/P00000001-AZ.zip](ftp://ftp.fec.gov/FEC/Presidential_Map/2016/P00000001/P00000001-AZ.zip)
### Description: 

### 0. Load data and modules

In [ ]:
#modules
import pandas as pd
import seaborn as sns; sns.set_style("whitegrid")
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline 
#data
file='P00000001-AZ.csv'; AZ_df=pd.read_csv(file,index_col=False)
# drop columns we won't need
AZ_df=AZ_df.drop(['cand_id','contbr_st','cand_id','receipt_desc',
            'receipt_desc','memo_text','form_tp','file_num','tran_id','memo_cd'],axis=1)

AZ_df.head(2)

### 1. Perform Sanity checks

In [ ]:
# Do I have any negative or zero contributions?

bol=AZ_df.contb_receipt_amt<0
plt.figure(); AZ_df[bol].contb_receipt_amt.plot(); plt.title('Negative')


bol=AZ_df.contb_receipt_amt==0
plt.figure()
AZ_df[bol].contb_receipt_amt.plot(); plt.title('Zero')

bol=AZ_df.contb_receipt_amt>0
plt.figure(); AZ_df[bol].contb_receipt_amt.plot();  plt.title('Positive')

# Remove all zero and negative data points

AZ_df=AZ_df[bol];

### 1. Extact information for Clinton and Trump

In [ ]:
names=AZ_df.cand_nm
names_bol=(names=='Trump, Donald J.') | (names=='Clinton, Hillary Rodham')
AZ_df=AZ_df[names_bol]

### 2. How many contributions did each candidate collect?

In [ ]:
sns.countplot(y="cand_nm",data=AZ_df);
plt.title("Total number of contributions collected by each candidate");
plt.xlabel("Total number of contributions");

### 2.What was the average contribution for each candidate ?

In [ ]:
sns.barplot(y="cand_nm", x="contb_receipt_amt", data=AZ_df,estimator=np.mean)
plt.title("Average contributions collected by each candidate");
plt.xlabel("Dollars");

### 3. What was the amount in dollas received by each candidate ?

In [ ]:
sns.barplot(y="cand_nm", x="contb_receipt_amt", data=AZ_df,estimator=np.sum)
plt.title("Average contributions collected by each candidate");
plt.xlabel("Dollars");

### 4. How much retired people contributed to each campaing?

In [ ]:
retired_bol=AZ_df.contbr_occupation=='RETIRED';
Info_req_bol=AZ_df.contbr_occupation=='INFORMATION REQUESTED';
other_bol= (AZ_df.contbr_occupation!='RETIRED') | (AZ_df.contbr_occupation!='INFORMATION REQUESTED');
AZ_df['Occupation']='Other';
AZ_df.Occupation[Info_req_bol]='Requested';
AZ_df.Occupation[retired_bol]='Retired';
sns.barplot(hue="Occupation", y="contb_receipt_amt",x='cand_nm',data=AZ_df,estimator=np.mean)

In [ ]:
AZ_df['Date']=pd.to_datetime(AZ_df.contb_receipt_dt);
df=AZ_df[['cand_nm','contb_receipt_amt','Date']];
df.head(2)

In [ ]:
df[df.cand_nm=='Trump, Donald J.']

In [ ]:
type(df.cumsum())

In [ ]:
from ggplot import *
ggplot(aes(x='Date', y='contb_receipt_amt'), data=df) + geom_line()

In [ ]:
df[df['cand_nm']=='Trump, Donald J.']

In [ ]:
gammas = sns.load_dataset("gammas")
ax = sns.tsplot(time="timepoint", value="BOLD signal",
                unit="subject", condition="ROI",
               data=gammas)

### 0. Load data and modules

In [ ]:
gammas.head(1)

### 0. Load data and modules

### 0. Load data and modules

### 0. Load data and modules

### 0. Load data and modules

### 0. Load data and modules

In [ ]:
from ggplot import *

ggplot(aes(x='date', y='beef'), data=meat) + geom_line()
    

In [ ]:
meat.head()

Pick a dataset - ideally the dataset for your Capstone. If for some reason you want to do this on a different data set, you can find one on Mode Analytics or Google's public data sets directory, or pick another one you like from elsewhere.

        Get going by asking the following questions and looking for the answers with some code and plots:

            Can you count something interesting?

            Can you find some trends (high, low, increase, decrease, anomalies)?

            Can you make a bar plot or a histogram?

            Can you compare two related quantities?

            Can you make a scatterplot?

            Can you make a time-series plot?

        Having made these plots, what are some insights you get from them? Do you see any correlations? Is there a hypothesis you would like to investigate further? What other questions do they lead you to ask?

        By now you’ve asked a bunch of questions, and found some neat insights. Is there an interesting narrative, a way of presenting the insights using text and plots from the above, that tells a compelling story? As you work out this story, what are some other trends/relationships you think will make it more complete?

Submission Requirements: Submit links to a Github repository containing a Jupyter notebook. The notebook should contain:

    The questions you asked
    The trends you investigated
    The resulting visualizations and conclusions.
